### Lead1 신호데이터 추출 및 .npy 저장
- ptb, sph 모두 완료
- ANNIE 1116 작업 완료

## PTB

In [2]:
import wfdb
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm

# 데이터 프레임 로드
path = './ptb-xl/ptb-xl/'
df = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')

# 파일 이름 리스트 변환
filenames = df['filename_hr'].tolist()
base_path = './ptb-xl/ptb-xl/'

# Lead I 신호를 저장할 배열 초기화
lead1_signals = []

# 각 파일에서 Lead I 신호 추출
for filename in tqdm(filenames):
    signal, meta = wfdb.rdsamp(base_path + filename)

    # 신호 데이터 타입을 float32로 변경
    signal = signal.astype(np.float32)
    
    lead1 = signal[:, 0]  # Lead I 신호 추출
    lead1_signals.append(lead1)

    # 사용하지 않는 변수 해제
    del signal, meta
    gc.collect()  # 가비지 컬렉터 실행


# 배열을 numpy 배열로 변환
lead1_signals = np.array(lead1_signals)

# 최종 배열 형태 확인
print("Final array shape:", lead1_signals.shape)

Final array shape: (21799, 5000)


In [3]:
np.save('./ptb-xl/ptb-xl/lead1.npy', lead1_signals)

## SPH

In [5]:
import glob
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm

# .h5 파일 목록 가져오기
filenames = sorted(glob.glob("./records.tar/SPH/*.h5"))

# 메타데이터 로드 및 파일명 열 추가
df = pd.read_csv("./records.tar/SPH/metadata.csv")
df['filename'] = filenames

# Lead I 신호를 저장할 배열 초기화
lead1_signals_sph = []

# 각 파일에서 Lead I 신호 추출
for filename in tqdm(df['filename']):
    with h5py.File(filename, 'r') as f:
        signal = f['ecg'][:].T

    lead1 = signal[:, 0]  # Lead I 신호 추출

    # 데이터 길이에 따른 처리
    if lead1.shape[0] == 5000:
        pass
    else:
        start_idx = np.random.randint(0, lead1.shape[0] - 4999)
        lead1 = lead1[start_idx:start_idx+5000]

    lead1_signals_sph.append(lead1)

# 배열을 numpy 배열로 변환
lead1_signals_sph = np.array(lead1_signals_sph)

# 최종 배열 형태 확인 및 저장
print("Final array shape:", lead1_signals_sph.shape)

  2%|▏         | 411/25770 [00:05<05:40, 74.52it/s]


KeyboardInterrupt: 

In [ ]:
np.save('./ptb-xl/ptb-xl/lead1_sph.npy', lead1_signals_sph)

## 데이터 확인

In [4]:
lead1_signals = np.load('./ptb-xl/ptb-xl/lead1.npy')
lead1_signals

array([[-0.115, -0.115, -0.115, ...,  0.21 ,  0.21 ,  0.21 ],
       [-0.015, -0.015, -0.015, ..., -0.055, -0.055, -0.055],
       [-0.035, -0.035, -0.035, ..., -0.04 , -0.04 , -0.04 ],
       ...,
       [ 0.09 ,  0.09 ,  0.09 , ..., -0.055, -0.055, -0.055],
       [-0.045, -0.045, -0.045, ...,  0.07 ,  0.07 ,  0.07 ],
       [-0.05 , -0.05 , -0.05 , ..., -0.07 , -0.07 , -0.07 ]],
      dtype=float32)

In [6]:
lead1_signals_sph = np.load('./ptb-xl/ptb-xl/lead1_sph.npy')
lead1_signals_sph

array([[ 2.161e-02,  2.161e-02,  2.080e-02, ...,  5.759e-02,  5.521e-02,
         5.042e-02],
       [-5.521e-02, -5.359e-02, -5.441e-02, ..., -5.521e-02, -5.679e-02,
        -5.759e-02],
       [ 0.000e+00,  4.002e-03, -4.002e-03, ...,  0.000e+00, -4.799e-03,
        -8.003e-03],
       ...,
       [ 2.560e-02,  2.080e-02,  1.759e-02, ..., -2.240e-02, -2.319e-02,
        -2.000e-02],
       [-6.401e-03,  8.001e-04,  1.040e-02, ...,  4.905e-01,  6.846e-01,
         8.618e-01],
       [-5.521e-02, -5.600e-02, -5.679e-02, ..., -3.760e-02, -3.680e-02,
        -3.519e-02]], dtype=float16)